In [1]:
import sys
sys.path.append('../ry')
from libry import *
from numpy import *

In [2]:
K = Config()
K.addFile("../rai-robotModels/pr2/pr2.g")
K.addFile("../models/tables.g")

K.addFrame("obj0", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 0 .15)>" )
K.addFrame("obj1", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .2 .15)>" )
K.addFrame("obj2", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .4 .15)>" )
K.addFrame("obj3", "table1", "type:ssBox size:[.1 .1 .2 .02] color:[1. 0. 0.], contact, logical={ object }, joint:rigid, Q:<t(0 .6.15)>" )
K.addFrame("tray", "table2", "type:ssBox size:[.15 .15 .04 .02] color:[0. 1. 0.], logical={ table }, Q:<t(0 0 .07)>" );
K.addFrame("", "tray", "type:ssBox size:[.27 .27 .04 .02] color:[0. 1. 0.]" )
V = K.view()

In [3]:
lgp = K.lgp("../models/fol-pickAndPlace.g")
lgp.nodeInfo()
# this writes the initial state, which is important to check:
# do the grippers have the gripper predicate, do all objects have the object predicate, and tables the table predicate? These need to be set using a 'logical' attribute in the g-file
# the on predicate should automatically be generated based on the configuration

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '<ROOT>',
 'path': '',
 'state': ' STATE_0(START_STATE) {(gripper pr2R), (gripper pr2L), (table table2), (object obj0), (object obj1), (object obj2), (object obj3), (table tray), (partOf table2 tray), (on obj0 table1), (on obj1 table1), (on obj2 table1), (on obj3 table1), }'}

In [4]:
lgp.getDecisions()
# This is also useful to check: inspect all decisions possible in this node, which expands the node.
# If there is no good decisions, the FOL rules are buggy

['(grasp pr2R obj0)',
 '(grasp pr2R obj1)',
 '(grasp pr2R obj2)',
 '(grasp pr2R obj3)',
 '(grasp pr2L obj0)',
 '(grasp pr2L obj1)',
 '(grasp pr2L obj2)',
 '(grasp pr2L obj3)']

In [5]:
lgp.walkToDecision(3)
lgp.nodeInfo()
# Using getDecisions and walkToDecision and walkToParent, you can walk to anywhere in the tree by hand

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.1, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '(grasp pr2R obj3)',
 'path': '(grasp pr2R obj3) ',
 'state': ' STATE_4(STATE_0) {(gripper pr2R), (gripper pr2L), (table table2), (object obj0), (object obj1), (object obj2), (object obj3), (table tray), (partOf table2 tray), (on obj0 table1), (on obj1 table1), (on obj2 table1), (on obj3 table1),  decision(grasp pr2R obj3), (grasped pr2R obj3), (held obj3), (busy pr2R),  komo(touch pr2R obj3), (stable pr2R obj3), }'}

In [6]:
lgp.viewTree()

In [7]:
lgp.walkToNode("(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray)")
lgp.nodeInfo()

{'boundsConstraints': [0.0, 0.0, 0.0, 0.0, 0.0],
 'boundsCost': [0.30000000000000004, 0.0, 0.0, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': ' STATE_15(STATE_9) {(gripper pr2R), (gripper pr2L), (table table2), (object obj0), (object obj1), (object obj2), (object obj3), (table tray), (partOf table2 tray), (on obj0 table1), (on obj1 table1), (on obj2 table1), (on obj3 table1), (grasped pr2L obj1), (held obj1), (busy pr2L), (stable pr2L obj1),  decision(place pr2R obj0 tray), (placed obj0 tray), (on obj0 tray),  komo(touch pr2R obj0), (notheld obj0),  block(INFEASIBLE grasp ANY obj0),  komo(above obj0 tray), (stableOn tray obj0), }'}

In [8]:
# at a node, you can compute bounds, namely BT.seq (just key frames), BT.path (the full path),
# and BT.setPath (also the full path, but seeded with the BT.seq result)
lgp.optBound(BT.seq, True)
lgp.nodeInfo()

{'boundsConstraints': [0.0, 0.0, 0.04162188560772702, 0.0, 0.0],
 'boundsCost': [0.30000000000000004, 0.0, 0.3156448371533603, 0.0, 0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': ' STATE_15(STATE_9) {(gripper pr2R), (gripper pr2L), (table table2), (object obj0), (object obj1), (object obj2), (object obj3), (table tray), (partOf table2 tray), (on obj0 table1), (on obj1 table1), (on obj2 table1), (on obj3 table1), (grasped pr2L obj1), (held obj1), (busy pr2L), (stable pr2L obj1),  decision(place pr2R obj0 tray), (placed obj0 tray), (on obj0 tray),  komo(touch pr2R obj0), (notheld obj0),  block(INFEASIBLE grasp ANY obj0),  komo(above obj0 tray), (stableOn tray obj0), }'}

In [9]:
komo = lgp.getKOMOforBound(BT.seq)
komo.display()

In [10]:
komo=0

In [11]:
lgp.optBound(BT.path, True)
lgp.nodeInfo()

{'boundsConstraints': [0.0,
  0.0,
  0.04162188560772702,
  0.038249342092987845,
  0.0],
 'boundsCost': [0.30000000000000004,
  0.0,
  0.3156448371533603,
  2.3257052260359012,
  0.0],
 'boundsFeasible': [1, 1, 1, 1, 1],
 'decision': '(place pr2R obj0 tray)',
 'path': '(grasp pr2R obj0) (grasp pr2L obj1) (place pr2R obj0 tray) ',
 'state': ' STATE_15(STATE_9) {(gripper pr2R), (gripper pr2L), (table table2), (object obj0), (object obj1), (object obj2), (object obj3), (table tray), (partOf table2 tray), (on obj0 table1), (on obj1 table1), (on obj2 table1), (on obj3 table1), (grasped pr2L obj1), (held obj1), (busy pr2L), (stable pr2L obj1),  decision(place pr2R obj0 tray), (placed obj0 tray), (on obj0 tray),  komo(touch pr2R obj0), (notheld obj0),  block(INFEASIBLE grasp ANY obj0),  komo(above obj0 tray), (stableOn tray obj0), }'}

In [12]:
lgp.viewTree()

In [14]:
# finally, the full multi-bound tree search (MBTS)
# you typically want to add termination rules, i.e., symbolic goals
print("THIS RUNS A THREAD. CHECK THE CONSOLE FOR OUTPUT. THIS IS GENERATING LOTS OF FILES.")
lgp.addTerminalRule("(on obj0 tray) (on obj1 tray) (on obj2 tray)")
lgp.run(2)

THIS RUNS A THREAD. CHECK THE CONSOLE FOR OUTPUT. THIS IS GENERATING LOTS OF FILES.


In [20]:
# wait until you have some number of solutions found (repeat executing this line...)
lgp.numSolutions()

0

In [9]:
# query the optimization features of the 0. solution
lgp.getReport(0, BT.seqPath)

[{'description': 'qItself:ALL',
  'scale': 0.01,
  'sos_value': 266.48765105458034,
  'type': 'sos'},
 {'feature': 'Transition',
  'scale': 1.0,
  'sos_value': 4.782265034423557,
  'type': 'sos'},
 {'description': 'QuaternionNorms',
  'scale': 3.0,
  'sos_value': 2.36774550422229e-09,
  'type': 'sos'},
 {'description': 'qItself:ALL',
  'scale': 0.1,
  'sos_value': 16.48785154058007,
  'target': [-0.3171423554873642,
   1.306948272451323,
   0.8185236650408605,
   0.10001259017623283,
   0.025351267614893144,
   0.0007581673958919039,
   -0.17387714526922404,
   0.7648057441364903,
   0.3772943713787953,
   0.5234999667115854,
   0.3620807546943853,
   -1.1468553431800832,
   1.021701406866571,
   -1.2736827819664338,
   -2.005442867107267,
   -1.6044356024681812,
   1.5065867663607198,
   -0.30886642950348514,
   -0.5087078394307857,
   -0.09061586961680453,
   0.4914155842964163,
   -0.00037388147252578343,
   -0.016968494056924157,
   -0.07741838836286874,
   0.5540570235751109,
   0

In [22]:
# get the KOMO object for the seqPath computation of the 0. solution
komo = lgp.getKOMO(0, BT.seqPath)

In [23]:
komo.displayTrajectory() #SOOO SLOOOW (TODO: add parameter for display speed)

In [14]:
# assign K to the 20. configuration of the 0. solution, check display
# you can now query anything (joint state, frame state, features)
X = komo.getConfiguration(20)
K.setFrameState(X) 

In [5]:
lgp.stop() #stops the thread... takes a while to finish the current job

In [19]:
lgp.run(2) #will continue where it stopped

In [ ]:
komo=0
lgp=0